In [2]:
import numpy as np
from matplotlib import pyplot as plt
import astropy.units as u
from astropy import constants as const
import pandas as pd
from astropy.table import Column
from astropy.time import Time
from scipy import interpolate as interp
import time
# from Photoz import photoz_tools as phtz
from tractor import *
from skimage.transform import downscale_local_mean
import scipy.optimize as opt
import matplotlib as mpl
from scipy.spatial.distance import cdist

import itertools
import matplotlib.pyplot as plt
from astropy.table import hstack
import os

import SPHEREx_ObsSimulator as SPobs
from SPHEREx_Simulator_Tools import SPHEREx_Logger, data_filename
import SPHEREx_InstrumentSimulator as SPinst
import SPHEREx_SkySimulator as SPsky
from pkg_resources import resource_filename

# survey_plan_file = 'spherex_survey_plan_march_2021.fits'
survey_plan_file = "/Users/gemmahuai/Desktop/CalTech/SPHEREx/Redshift/spherex_survey_plan_R2.fits"
SPHEREx_Pointings = SPobs.Pointings(input_file = survey_plan_file,
                                   Gaussian_jitter=1.8, 
                                   roll_angle='psi2')

from spherex_parameters import load_spherex_parameters
# Load instrument and project parameters as a dictionary
spherex_parameters = load_spherex_parameters()


ds1 = 4
ds2 = 2
trim = 32
SPHEREx_Instrument = SPinst.Instrument(
    instrument_data=spherex_parameters,
    psf=data_filename("psf/simulated_PSF_database_centered_v3_og.fits"),
    psf_downsample_by_array={1: ds1, 2: ds1, 3: ds1, 4: ds2, 5: ds2, 6: ds2},
    psf_trim_by_array={1: trim, 2: trim, 3: trim, 4: trim, 5: trim, 6: trim},

    noise_model=SPinst.white_noise,
    dark_current_model=SPinst.poisson_dark_current,
    lvf_model=SPinst.Tabular_Bandpass()
)

from SPHEREx_SkySimulator import QuickCatalog
from SPHEREx_SkySimulator import Catalog_to_Simulate
# path='/Users/zhaoyuhuai/SPHEREx-Sky-Simulator/docs/QuickCatalog/'

from pyarrow import parquet
from astropy.table import Table
from astropy.io import fits
Channels = Table.read(data_filename('Channel_Definition_03022021.fits'))
Scene = SPsky.Scene(SPHEREx_Pointings,
                        zodi_model=SPsky.zodicalc.ModifiedKelsallModelWithHPFT())

COSMOS_tab = Table.read('/Users/gemmahuai/Desktop/CalTech/SPHEREx/SPHEREx_2023/COSMOS2020_FARMER_R1_v2.1_p3_in_Richard_sim_2023Dec4.fits', format='fits')
COSMOS_sim_sources = Table.read('/Users/gemmahuai/Desktop/CalTech/SPHEREx/SPHEREx_2023/COSMOS2020_FARMER_R1_v2.1_p3_in_Richard_sim_2023Dec4.fits', format='fits')



# Ari's PSF functionalities

In [5]:
psf_SkySim = SPinst.PSF(data_filename("psf/simulated_PSF_database_centered_v3_og.fits"))

In [13]:
psf_SkySim.get_psf_data(1) # 54 total PSFs in the fits file; 9 per array

array([[ 1.11691683e-08,  2.15529649e-08,  9.03630175e-09, ...,
         3.53132774e-09,  1.08916891e-09,  5.97595655e-11],
       [ 1.95822964e-08,  1.78299551e-08,  3.03810004e-09, ...,
         7.11717329e-09,  4.03030302e-09,  3.28106685e-10],
       [ 8.33344004e-09,  2.42356217e-09,  2.29899170e-09, ...,
         4.55492596e-09,  7.23745538e-09,  1.37387079e-09],
       ...,
       [ 1.21280070e-10, -5.10280195e-12,  5.90458217e-10, ...,
         1.13268702e-09,  5.46466870e-10,  2.58894702e-10],
       [ 4.25289498e-09,  2.77592987e-09,  1.96479907e-09, ...,
         1.32692656e-09,  2.55221863e-10,  3.03665154e-11],
       [ 9.72908782e-10,  6.88862642e-10,  3.82960226e-10, ...,
         2.60069740e-10,  3.22737547e-11, -2.03186813e-13]], dtype='>f8')

In [16]:
psf_SkySim.psf_y

array([  0. ,  16.2, -16.2, -16.2,  16.2,   0. ,   0. ,  16.2, -16.2,
         0. ,   0. ,  15. ,  15. , -15. , -15. ,  15. ,   0. , -15. ,
         0. ,  16.2, -16.2,   0. , -16.2,   0. ,  16.2, -16.2,  16.2,
        15. , -15. ,  15. ,   0. , -15. ,  15. ,   0. ,   0. , -15. ,
        15. , -15. ,   0. ,   0. , -15. ,  15. , -15. ,   0. ,  15. ,
         0. , -15. ,  15. ,   0. , -15. , -15. ,  15. ,  15. ,   0. ])

In [18]:
psf_fn = data_filename("psf/simulated_PSF_database_centered_v3_og.fits")

In [28]:
psf_fn

'/Users/gemmahuai/anaconda3/envs/spherexsim_tractor/lib/python3.11/site-packages/SPHEREx_Simulator_Data/psf/simulated_PSF_database_centered_v3_og.fits'

In [27]:
with fits.open(psf_fn,mode='readonly') as f:
    print(len(f))
    f0 = f[1]
    data = f0.data

55


In [26]:
data.shape

(401, 401)